#  INIT

In [1]:
import sys
sys.path.append('../../')
sys.path.append('../../infras/')
sys.path.append('../../infras/cellMix/')
sys.path.append('../../infras/dashboards/')
sys.path.append('../../infras/cytof_data/')
sys.path.append('../../preprocess/intra_variance/')



from aggregate_intra_variance import AggregateIntraVariance
from pp_clean_high_intra_var import PpCleanHighIntraVar
from cellMix_coordinator import CellMixCoordinator
from cytof_cell_count_infra import CytofCellCountInfra

import exploration_plots
import dashs

%matplotlib inline

In [2]:
cluster_info_path= r"../../infras/cytof_data/raw_data/CyTOF.features.and.clusters.info.xlsx"
cytof_data_path=r"../../infras/cytof_data/raw_data/filtered.esetALL.CyTOF.abundance.only.xlsx"

cci = CytofCellCountInfra(cluster_info_path,cytof_data_path)


In [3]:
import pandas as pd
import numpy as np
from functools import partial
import multiprocessing

from itertools import chain
from scipy.optimize import least_squares
from sklearn.metrics import mean_squared_error
from functools import partial
from scipy.optimize import minimize
import scipy.optimize
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
import math


from scipy.stats import entropy
from functools import reduce
import matplotlib.patches as patches


import os

In [4]:
from deconv_py.preprocess.base import BasePreprocess as PP_base
from deconv_py.preprocess.cell_specific import CellSpecific as PP_proteins

from deconv_py.infras.data_factory import DataFactory
from deconv_py.infras.data_loader import DataLoader

from deconv_py.models.base import Base as Models_base
from deconv_py.models.cell_proportions_models import CellProportions
from deconv_py.models.cell_specific_models import CellSpecificPerPermutation

from deconv_py.experiments.cell_specific import CellSpecificMetricsPlot

# from deconv_py.infras.cellMix.cellMix_coordinator import CellMixCoordinator

In [5]:
data_loader = DataLoader(mass_spec_path=os.path.abspath('../../data/20190801_filtered_imputed_data.csv'),
                         protein_profile_path = os.path.abspath('../../data/20150718_Cerberus_proteinGroups.txt'),as_csv = True)
data_factory = DataFactory(data_loader)
ms = data_loader.get_mass_spec_data()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: DtypeWarning: Columns (5,6,2764,2765,2773,2774,2775,2776) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
from deconv_py.preprocess.cell_specifics.pp_clean_irrelevant_proteins import PpCleanIrrelevantProteins
from deconv_py.preprocess.cell_specifics.pp_entropy_based import PpEntropyBased
from deconv_py.models.cell_proportions.basic import BasicDeconv

In [7]:
%connect_info

{
  "shell_port": 55013,
  "iopub_port": 55014,
  "stdin_port": 55015,
  "control_port": 55016,
  "hb_port": 55017,
  "ip": "127.0.0.1",
  "key": "e008e935-571cc9d86f6c06273d2f43cd",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-f1805000-935d-43c4-ba8f-d064f17890d8.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


# read data and cytof

## read protein profile and mixtures 

**LFQ:** rel. quantification – for same protein across different samples

**iBAQ:** orthogonal comparison – for different proteins in the same sample

**Intensity:** supposed  to be both


In [8]:
A_intensity,B_intensity = data_factory.load_IBD_all_vs("LFQ intensity",index_func=lambda x:x.split(';')[0],log2_transformation=True,auto_filter_by = True)

# #filter intraVar
filt = PpCleanHighIntraVar(how="std")
A_intensity,B_intensity = filt.transform([A_intensity,B_intensity])

#agg intravar
agg_iv = AggregateIntraVariance(how="median")
A_intensity,_ = agg_iv.transform([A_intensity,None])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (5,6,2764,2765,2773,2774,2775,2776) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


## read Cytof data per profile matrix (because of label propagation )

In [9]:
A_intensity_Cytof_cell,cytof_count_of_A_intensity = cci.return_mass_and_cytof_not_none_cells_counts(A_intensity.T,cytof_label_prop_A_ints)
A_intensity_Cytof_cell = A_intensity_Cytof_cell.T

In [28]:
high_low_limits_for_cytof = {
    "B cells ":[0.05,0.11],
    "CD4+ effector memory T cells":[0.17,0.26],
    "CD8+ effector memory T cells":[0.08,0.18],
    "Monocytes":[0.15,0.33],
    "NK":[0.07,0.13],
    "Na?ve CD4 Tcell":[0.17,0.27],
    "Na?ve CD8 Tcell":[0.055,0.150],
    "Plasmacytoid dendritic cells":[0.01,0.02]
}

In [38]:
new_list = [] 
for cell,samples_values in cytof_count_of_A_intensity.iterrows():
    low_trh,high_trh = high_low_limits_for_cytof[cell] 
    new_list.append(samples_values.map(lambda x: 0 if x < low_trh else ( 2 if x > high_trh else 1) ))
quantified_cytof_count = pd.concat(new_list,axis=1).T


mixtures,HR-38-V1,HR-42-V1,HR-44-V1,HR-46-V1,HR-47-V1,HR-48-V1,HR-29-V1,HR-30-V1,HR-31-V1,HR-32-V1,...,HR-37-V1,HR-20-V1,HR-21-V1,HR-22-V1,HR-23-V1,HR-24-V1,HR-26-V1,HR-27-V1,HR-40-V1,HR-28-V1
B cells,1 / 0.05748913125313689,0 / 0.040475752657773846,1 / 0.07382117139940213,0 / 0.033409372494428706,1 / 0.0963654048075018,1 / 0.07088535371444636,1 / 0.08802035802573942,1 / 0.07737267074885844,1 / 0.10568696868010552,0 / 0.029982827736034307,...,2 / 0.13093460251767375,0 / 0.02597897063029539,2 / 0.1389467093919956,0 / 0.0276735567035701,2 / 0.1785472066162758,2 / 0.12180077814182609,1 / 0.06995751144760895,1 / 0.09281158362181395,1 / 0.08882781669740464,0 / 0.04759066643838731
CD4+ effector memory T cells,2 / 0.26650976990051906,1 / 0.2172162736043859,1 / 0.2161514302503502,1 / 0.18006687358756931,1 / 0.23179916646855125,1 / 0.2024491379761728,1 / 0.18334539952938986,2 / 0.27017713226554435,0 / 0.1649415876681181,2 / 0.3051797235161402,...,1 / 0.19706361704380684,1 / 0.2030061048968371,2 / 0.27827162051521254,1 / 0.2172630319762457,1 / 0.24324863188336499,2 / 0.28871314873199894,2 / 0.2811121657484114,0 / 0.16063025228516234,1 / 0.18774070306395407,1 / 0.1826388067493326
CD8+ effector memory T cells,1 / 0.1286899344138467,1 / 0.10762435647113666,1 / 0.14156711506619848,0 / 0.0665953633489112,1 / 0.09944821408030692,1 / 0.1485288288010573,0 / 0.06487638570543956,2 / 0.21255693633222855,0 / 0.06977204348290543,2 / 0.1885071362444282,...,0 / 0.07062828839971375,0 / 0.07253218380919425,1 / 0.12957010884857056,0 / 0.07157108096341289,0 / 0.07635113831296715,1 / 0.15806967992466267,1 / 0.13220611146808867,0 / 0.05401738000153972,1 / 0.12818671943298932,1 / 0.14664693142115623
Monocytes,1 / 0.30134554950974546,0 / 0.13854130799088774,2 / 0.3913716136170978,1 / 0.2645659757775016,1 / 0.17362865220810555,1 / 0.18365215736295695,1 / 0.16883680693975722,0 / 0.14958636869377073,1 / 0.25044861164515697,1 / 0.1990442298035325,...,2 / 0.3684252498431328,1 / 0.21935514886451016,1 / 0.18638930732344297,2 / 0.48555398493344326,0 / 0.07805842346484157,1 / 0.23323071606401893,0 / 0.09669512044507556,1 / 0.26109690418100256,0 / 0.11806361497819358,0 / 0.09184053219513967
NK,1 / 0.09103586936953449,1 / 0.0897647228753245,1 / 0.08942927027871164,1 / 0.10486732984023321,0 / 0.019828510544361397,1 / 0.12276469310446218,0 / 0.03391976841762079,1 / 0.11492078441346995,2 / 0.13528788290613095,0 / 0.048754172562082046,...,0 / 0.04956036614105976,0 / 0.048773197379843036,0 / 0.052065291420118724,1 / 0.0886838580379745,0 / 0.036150025401959746,0 / 0.04658816562106499,2 / 0.21875859790526725,0 / 0.04516773666312552,0 / 0.04500625882372149,0 / 0.05473660700620906
Na?ve CD4 Tcell,0 / 0.09263555364376509,1 / 0.26953623962950196,0 / 0.05744007943364349,1 / 0.23065964031576044,1 / 0.19103007575387962,1 / 0.2181106672530049,1 / 0.2508391582881916,0 / 0.142878151780313,1 / 0.19703411792682415,1 / 0.18069052893847223,...,0 / 0.11905581385507383,2 / 0.30293509804808494,0 / 0.11562963460311461,0 / 0.07732038167264114,1 / 0.20602848596588033,0 / 0.10074968402805468,0 / 0.154482316936358,2 / 0.2888390671158988,1 / 0.26845132969612906,2 / 0.282396354801551
Na?ve CD8 Tcell,0 / 0.04958864594406973,1 / 0.12155266847277732,0 / 0.003965485004748802,1 / 0.1016379058716012,2 / 0.18326905073551897,0 / 0.03948242445694308,2 / 0.19979435516816071,0 / 0.020509132220264677,1 / 0.07023585833397177,0 / 0.036391309259131095,...,0 / 0.03683132196031781,1 / 0.11423454127897731,1 / 0.0824864512421916,0 / 0.029262261568973724,2 / 0.16743711313161128,0 / 0.03384783151255674,0 / 0.03244045380866832,1 / 0.08366043054950674,2 / 0.1516350006225985,2 / 0.1812969126490415
Plasmacytoid dendritic cells,1 / 0.012705545965382774,1 / 0.015288678298212073,2 / 0.02625383494984744,1 / 0.01819753876399426,0 / 0.004630925401774666,1 / 0.014126737330956407,1 / 0.01036776792570081,1 / 0.011998823545550295,0 / 0.0065929293567869515,1 / 0.011450071940179243,...,2 / 0.02750074023922161,1 / 0.013184755092257775,1 